In [ ]:
import os
import urllib3
import zipfile

import pandas as pd

# Tratamiento de datos

En este notebook preparamos la información de usuarios, items y ratings que vamos a utilizar en el experimento. El usuario sólo tiene que escribir la cantidad de películas más valoradas y usuarios más activos que desea utilizar. Los datos se obtendrán del dataset de MovieLens.

## Input Datos

Indicar cuantos usuarios y películas se van a usar en el experimento.

In [2]:
num_users = 100
num_movies = 100

## Preparación de entorno y Descarga de dataset

A continuación, preparamos el entorno para ejecutar los experimentos, descargamos el dataset y descomprimimos el dataset en la carpeta `data`.

In [3]:
# Creamos las carpetas que no existan
if not os.path.exists('data'):
    os.mkdir('data')
    os.mkdir('data/original_dataset')
    os.mkdir('data/experiment_data')
    os.mkdir('data/matrices_data')
    
# Descargamos dataset
url_dataset = 'http://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
path_dataset = 'data/original_dataset/ml-latest-small.zip'

http = urllib3.PoolManager()
request = http.request('GET', url_dataset, preload_content=False)

chunk_size = 100

with open(path_dataset, 'wb') as out:
    while True:
        data = request.read(chunk_size)
        if not data:
            break
        out.write(data)

request.release_conn()

# Descomprimimos
zip_ref = zipfile.ZipFile(path_dataset, 'r')
zip_ref.extractall('data/original_dataset')
zip_ref.close()

# Borramos el fichero zip
os.remove('data/original_dataset/ml-latest-small.zip')

## Obtenemos los ratings del experimento

Ahora, cargamos todos los ratings y nos quedamos con aquellos de los usuarios con más actividad y de los items con más ratings.

In [4]:
all_ratings_DF = pd.read_csv('data/original_dataset/ml-latest-small/ratings.csv')
all_ratings_DF.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
# Buscamos los X usuarios más activos
users_id = all_ratings_DF.groupby(by='userId').count().sort_values('rating', ascending=False)[:num_users].index.values

# Buscamos los X items más valorados
movies_id = all_ratings_DF.groupby(by='movieId').count().sort_values('rating', ascending=False)[:num_movies].index.values

In [6]:
# Filtramos de los ratings aquellos usuarios e items que no están en las listas calculadas antes
final_ratings = all_ratings_DF[all_ratings_DF['userId'].isin(users_id)]
final_ratings = final_ratings[final_ratings['movieId'].isin(movies_id)]

In [7]:
# Guardamos los ratings filtrados
final_ratings.to_csv('data/experiment_data/ratings.csv', index=False)

## Siguiente Notebook

Ya tenemos el dataset del experimento preparado. A continuación, hay que descargar toda la información posible de las películas. Para ello, ejecute el notebook `extraer_informacion_peliculas`